<a href="https://colab.research.google.com/github/dntlr4463/coding/blob/main/%EC%9E%AC%EB%AC%B4%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests 
import pandas as pd 
import io 
import zipfile
import xml.etree.ElementTree as et 
import json
from openpyxl import load_workbook
import numpy as np
import os
import pymysql
from sqlalchemy import create_engine
import MySQLdb


crtfc_key = '9980209450e0c80afafc2496c87155d659bf0c17'

def get_corpcode(crtfc_key): 
    """ 
    OpenDART 기업 고유번호 받아오기 
    return 값: 주식코드를 가진 업체의 DataFrame 
    """ 
    params = {'crtfc_key':crtfc_key} 
    items = ["corp_code","corp_name","stock_code","modify_date"] 
    item_names = ["고유번호","회사명","종목코드","수정일"] 
    url = "https://opendart.fss.or.kr/api/corpCode.xml" #요청 url
    res = requests.get(url,params=params) #url 불러오기
    zfile = zipfile.ZipFile(io.BytesIO(res.content))  #zip file 받기
    fin = zfile.open(zfile.namelist()[0])  #zip file 열고
    root = et.fromstring(fin.read().decode('utf-8'))  #utf-8 디코딩
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: # 종목코드가 있는 경우 
            data.append([]) #data에 append하라 
            for item in items: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns=item_names) 
    return df

def Frame(url, items, item_names, params): 
    """
    url : json형태로 요청하는 주소
    items : 반환되는 데이터들의 key를 가진 리스트
    item_names :  데이터프레임을 만들때 컬럼명 리스트
    params : url 요청시 필수값으로 들어가는 인자들을 가진 딕셔너리
    """
    res = requests.get(url, params)
    json_data = res.json()
    json_dict = json.loads(res.text) 
    data = [] 
    if json_dict['status'] == "000":  # 오류 없이 정상적으로 데이터가 있다면 
        for line in json_dict['list']: 
            data.append([])
            for itm in items: 
                if itm in line.keys(): 
                    data[-1].append(line[itm]) 
                else: 
                    data[-1].append('')
    df = pd.DataFrame(data, columns=item_names)
    return df

def get_단일회사재무제표(crtfc_key, corp_code, bsns_year, reprt_code, fs_div = "CFS"): 
    items = ["rcept_no","reprt_code","bsns_year","corp_code","sj_div","sj_nm", 
             "account_id","account_nm","account_detail","thstrm_nm", "thstrm_amount",
             "thstrm_add_amount","frmtrm_nm","frmtrm_amount", "frmtrm_q_nm","frmtrm_q_amount",
             "frmtrm_add_amount","bfefrmtrm_nm", "bfefrmtrm_amount","ord"] 
    item_names = ["접수번호","보고서코드","사업연도","고유번호","재무제표구분", "재무제표명",
                  "계정ID","계정명","계정상세","당기명","당기금액", "당기누적금액","전기명","전기금액","전기명(분/반기)", 
                  "전기금액(분/반기)","전기누적금액","전전기명","전전기금액", "계정과목정렬순서"] 
    params = {'crtfc_key':crtfc_key, 'corp_code':corp_code, 'bsns_year':bsns_year, 'reprt_code':reprt_code, 'fs_div':fs_div} 
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?" 
    df = Frame(url, items, item_names, params)
    return df

def sep(x, df): 
    df = df[df['재무제표구분'] == x]
    df = df.drop(['계정ID', '계정상세', '당기누적금액', '전기명(분/반기)', '전기금액(분/반기)', '전기누적금액', '계정과목정렬순서'], axis = 'columns')
    return df
 
    final_BS = sep('BS', final_reprt_df) #재무상태표 
    final_CIS = sep('CIS', final_reprt_df) #포괄손익계산서 
    final_CF = sep('CF', final_reprt_df) #현금흐름표
    final_SCE = sep('SCE', final_reprt_df) #자본변동표
    final_IS = sep('IS', final_reprt_df) #손익계산서
    
def filtering(df):
    df = df.replace('', np.NaN)
    df = df.drop_duplicates(['계정명'], keep = 'first')
    df['당기금액'] = df['당기금액'].astype(float)
    df['전기금액'] = df['전기금액'].astype(float)
    df['전전기금액'] = df['전전기금액'].astype(float)
    return df
    
def sql_db(corp_code):
    mysqldb = pymysql.connect(host='localhost', user='root', password='dntlr59531!', db='qw_test', charset='utf8mb4')
    cursor = mysqldb.cursor(pymysql.cursors.DictCursor)

    # 기업마다 작성 방식이 다르므로 하나씩 연산되는 것을 확인해야함.
    liq_asset = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '유동자산' and 고유번호 = '{corp_code}'"
    liq_debt = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '유동부채' and 고유번호 = '{corp_code}'"
    liq_inventory = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '재고자산' and 고유번호 = '{corp_code}'"
    cash = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '현금및현금성자산' and 고유번호 = '{corp_code}'"
    Total_asset = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '자산총계' and 고유번호 = '{corp_code}'"
    Total_capital= f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '자본총계' and 고유번호 = '{corp_code}'"
    Total_debt = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '부채총계' and 고유번호 = '{corp_code}'"
    #Total_dc = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '부채 및 자본 총계' and 고유번호 = '{corp_code}'"
    Total_dc = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '자본과부채총계' and 고유번호 = '{corp_code}'"
    long_borrowed = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '사채 및 장기차입금' and 고유번호 = '{corp_code}'"
    short_borrowed = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '사채 및 차입금' and 고유번호 = '{corp_code}'"
    non_current_assets = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '비유동자산' and 고유번호 = '{corp_code}'"
    non_current_debt = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '비유동부채' and 고유번호 = '{corp_code}'"

    ebit = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 like '영업이익%' and 고유번호 = '{corp_code}'"
    #ebit = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 like '영업손익%' and 고유번호 = '{corp_code}'"

    interest_expense = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '이자비용' and 고유번호 = '{corp_code}'"
    interest_income = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '이자수익' and 고유번호 = '{corp_code}'"
    depreciation = f"SELECT 당기금액, 전기금액, 전전기금액 FROM qw_test.bs_timeseries WHERE 계정명 = '감가상각비' and 고유번호 = '{corp_code}'"
    
    col_list = [{'당기금액':0, '전기금액':0, '전전기금액':0}] # NULL DATA 처리

    cursor.execute(liq_asset)
    result_liq_asset = cursor.fetchall()
    df_liq_asset = pd.DataFrame(result_liq_asset)

    cursor.execute(liq_debt)
    result_liq_debt = cursor.fetchall()
    df_liq_debt = pd.DataFrame(result_liq_debt)
    
    cursor.execute(liq_inventory)
    result_liq_inventory = cursor.fetchall()
    df_liq_inventory = pd.DataFrame(result_liq_inventory)
    #df_liq_inventory = pd.DataFrame(col_list) # NULL일 경우 이 코드를 이용하여 풀면됨

    cursor.execute(cash)
    result_cash = cursor.fetchall()
    df_cash = pd.DataFrame(result_cash)

    cursor.execute(Total_asset)
    result_asset = cursor.fetchall()
    df_asset = pd.DataFrame(result_asset)

    cursor.execute(Total_capital)
    result_capital = cursor.fetchall()
    df_capital = pd.DataFrame(result_capital)

    cursor.execute(Total_debt)
    result_debt = cursor.fetchall()
    df_debt = pd.DataFrame(result_debt)

    cursor.execute(Total_dc)
    result_dc = cursor.fetchall()
    df_dc = pd.DataFrame(result_dc)

    cursor.execute(long_borrowed)
    result_long_borrowed = cursor.fetchall()
    df_long_borrowed = pd.DataFrame(result_long_borrowed)

    cursor.execute(short_borrowed)
    result_short_borrowed = cursor.fetchall()
    df_short_borrowed = pd.DataFrame(result_short_borrowed)

    cursor.execute(ebit)
    result_ebit = cursor.fetchall()
    df_ebit = pd.DataFrame(result_ebit)

    cursor.execute(interest_expense)
    result_interest_expense = cursor.fetchall()
    df_interest_expense = pd.DataFrame(result_interest_expense)

    cursor.execute(interest_income)
    result_interest_income = cursor.fetchall()
    df_interest_income = pd.DataFrame(result_interest_income)

    cursor.execute(depreciation)
    result_depreciation = cursor.fetchall()
    df_depreciation = pd.DataFrame(result_depreciation)

    cursor.execute(non_current_assets)
    result_non_current_assets = cursor.fetchall()
    df_non_current_assets = pd.DataFrame(result_non_current_assets)
    
    cursor.execute(non_current_debt)
    result_non_current_debt = cursor.fetchall()
    df_non_current_debt = pd.DataFrame(result_non_current_debt)
    
    # Calculation
    liquidity_ratio = df_liq_asset/df_liq_debt*100
    quick_ratio = (df_liq_asset-df_liq_inventory.fillna(0))/df_liq_debt*100
    cash_ratio = df_cash/df_liq_debt*100
    nwct_ratio = (df_liq_asset-df_liq_debt)/df_asset*100
    debt_ratio = df_debt/df_capital*100
    #debt_ratio = df_debt/(df_dc-df_debt)*100
    capital_ratio = df_capital/(df_debt+df_capital)*100
    #capital_ratio = (df_dc-df_debt)/df_dc*100
    dependence_debt = (df_long_borrowed+df_short_borrowed)/df_liq_asset*100
    average_interest_rate_borrowings = df_interest_expense/(df_long_borrowed+df_short_borrowed)*100
    interest_c_ratio = df_ebit/df_interest_expense
    net_interest_coverage_ratio = df_ebit/(df_interest_expense-df_interest_income)
    ebit_depreciation_ratio = (df_ebit+df_interest_expense+df_depreciation)/df_interest_expense*100
    non_current_ratio = df_non_current_assets/df_capital*100
    #non_current_ratio = df_non_current_assets/(df_dc-df_debt)*100
    long_term_fitness_rate = df_non_current_assets/(df_capital+df_non_current_debt)*100
    #long_term_fitness_rate = df_non_current_assets/(df_dc-df_debt+df_non_current_debt)*100

    col = ['present', 'prior', 'priorprior']
    liquidity_ratio.columns = col
    quick_ratio.columns = col
    cash_ratio.columns = col
    nwct_ratio.columns = col
    debt_ratio.columns = col
    capital_ratio.columns = col
    dependence_debt.columns = col
    average_interest_rate_borrowings.columns = col
    interest_c_ratio.columns = col
    net_interest_coverage_ratio.columns = col
    ebit_depreciation_ratio.columns = col
    non_current_ratio.columns = col
    long_term_fitness_rate.columns = col

    liquidity_ratio.loc[:,'analysis'] = ['유동성비율']
    quick_ratio.loc[:,'analysis'] = ['당좌비율']
    cash_ratio.loc[:,'analysis'] = ['현금비율']
    nwct_ratio.loc[:,'analysis'] = ['순운전자본비율']
    debt_ratio.loc[:,'analysis'] = ['부채비율']
    capital_ratio.loc[:, 'analysis'] = ['자기자본비율']
    dependence_debt.loc[:, 'analysis'] = ['차입금의존도']
    average_interest_rate_borrowings.loc[:, 'analysis'] = ['차입금평균이자율']
    interest_c_ratio.loc[:, 'analysis'] = ['이자보상비율']
    net_interest_coverage_ratio.loc[:, 'analysis'] = ['순이자보상비율']
    ebit_depreciation_ratio.loc[:, 'analysis'] = ['EBITDA/이자비용비율']
    non_current_ratio.loc[:, 'analysis'] = ['비유동비율']
    long_term_fitness_rate.loc[:, 'analysis'] = ['비유동장기적합률']
    

    financial_analysis = pd.concat([liquidity_ratio, quick_ratio, cash_ratio, nwct_ratio,
                                    debt_ratio, capital_ratio, dependence_debt, average_interest_rate_borrowings,
                                    interest_c_ratio, net_interest_coverage_ratio, ebit_depreciation_ratio, non_current_ratio,
                                   long_term_fitness_rate], axis=0, ignore_index = True)
    
    d_corp_code = [f'{corp_code}']
    df_corp_code = pd.DataFrame(d_corp_code, columns = ['corp_code'])
    financial_analysis = pd.concat([financial_analysis, df_corp_code], axis = 1)   
    financial_analysis['corp_code'] = financial_analysis['corp_code'].fillna(f'{corp_code}')
    
    return financial_analysis

In [ ]:
# 1분기 : 11013
# 반기 : 11012
# 3분기 : 11014
# 시업보고서 : 11011
list_name = ['더블유게임즈', '데브시스터즈', '미투온', '티쓰리', '액션스퀘어',
             '미스터블루', '썸에이지', '넵튠', '넥슨게임즈', '넷마블',
             '크래프톤', '펄어비스', '카카오게임즈', '베스파', '밸로프',
             '모비릭스', '미투젠', '드래곤플라이', '스카이문스테크놀로지','엔씨소프트',
             '네오위즈홀딩스', '한빛소프트', '액토즈소프트', '엠게임','룽투코리아',
             '컴투스홀딩스', '조이시티', '웹젠', '컴투스','네오위즈',
             '위메이드맥스', '위메이드', '위메이드플레이'
            ]


def load_corp_code():
    corp_code = stock_comp[stock_comp['회사명']==list_name[28]].고유번호
    return corp_code
    
def load_data(corp_code):
    final_reprt_df = get_단일회사재무제표('9980209450e0c80afafc2496c87155d659bf0c17', corp_code, 2022, '11011', fs_div = "CFS")
    return final_reprt_df

def post_bs_frame(df):

  pymysql.install_as_MySQLdb()
  engine = create_engine("mysql+mysqldb://root:pw@localhost:3306/qw_test", encoding='utf-8')
  conn = engine.connect()
  df.to_sql(name='table_name', con=engine, if_exists='append', index=False)

def post_fa_frame(df):

  pymysql.install_as_MySQLdb()
  engine = create_engine("mysql+mysqldb://root:pw@localhost:3306/qw_test", encoding='utf-8')
  conn = engine.connect()
  df.to_sql(name='table_name', con=engine, if_exists='append', index=False)


def post_mn_frame(df):
  
  pymysql.install_as_MySQLdb()
  engine = create_engine("mysql+mysqldb://root:pw@localhost:3306/qw_test", encoding='utf-8')
  conn = engine.connect()
  df.to_sql(name='table_name', con=engine, if_exists='append', index=False)


In [ ]:
final_data = load_data(load_corp_code())
bs_df = sep('BS', final_data)
cis_df = sep('CIS', final_data)
cf_df = sep('CF', final_data)

In [ ]:
bs_df = filtering(bs_df)
cis_df = filtering(cis_df)
cf_df = filtering(cf_df)

In [ ]:
post_bs_frame(bs_df)
post_bs_frame(cis_df)
post_bs_frame(cf_df)

In [ ]:
df = sql_db('고유번호입력하기')
df

In [ ]:
post_fa_frame(df)

In [ ]:
def mean_frame():
  mysqldb = pymysql.connect(host='localhost', user='root', password='pw', df='qw_test', charset='utf8mb4')
  cursor = mysqldb.cursor(pymysql)

  list_analysis = ['유동성비율', '당좌비율', '현금비율', '순운전자본비율', '부채비율',
                   '자기자본비율', '차입금의존도', '차입금평균이자율', '이자보상비율', '순이자보상비율',
                   'EBITDA/이자비용비율', '비유동비율', '비유동장기적합률']

  ratio = f"select * from qw_test.table where analysis = '{list_analysis[i]""

  cursor.execute(ratio)
  result_ratio = cursor.fetchall()
  df_ratio = pd.DataFrame(result_ratio)

  value = pd.DataFrame(df_ratio.mean())
  mean_value = value.transpose()
  mean_value = mean_value.drop([mean_value.columns[0], axis=1])
  mean_value.columns = ['present_mean', 'prior_mean', 'priorprior_mean']

  d_analysis = [f'{list_analysis[i]']
  df_analysis = pd.DataFrame(d_analysis, columns = ['mean_analysis'])
  financial_mean = pd.concat([mean_value, df_analysis], axis = 1)
  financial_mean

  return financial mean

In [ ]:
mean_frame()

In [ ]:
post_mn_frame(mean_frame())

In [ ]:
def condition_data(corp_data):
  mysqldb = pymysql.connect(host='localhost', user='root', password='pw', df='qw_test', charset='utf8mb4')
  cursor = mysqldb.cursor(pymysql)

  company = f"select * from financial_analysis where cotp_code = '{corp_code}'"
  mean = "select * from financial_mean"

  cursor.execute(company)
  result_company = cursor.fetchall()
  df_company = pd.DataFrame(result_company)

  cursor.execute(mean)
  result_mean = cursor.fetchall()
  df_mean = pd.DataFrame(result_mean)

  df_compare['present_condition'] = df_compare['present']>=df_compare['present_mean']
  df_compare['prior_condition'] = df_compare['prior']>=df_compare['prior_mean']
  df_compare['priorprior_condition'] = df_compare['priorprior']>=df_compare['priorprior_mean']

  list_inverse = ['부채비율', '차입금의존도', '차입금평균이자율', '비유동비율','비유동장기적합률']

  df_r0 = df_compare[df_compare['analysis']==list_inverse[0]].replace(True, '위험')
  df_r0 = df_r0[df_r0['analysis']==list_inverse[0].replace(False, '양호')]
  df_r1 = df_compare[df_compare['analysis']==list_inverse[1]].replace(True, '위험')
  df_r1 = df_r0[df_r0['analysis']==list_inverse[1].replace(False, '양호')]
  df_r2 = df_compare[df_compare['analysis']==list_inverse[2]].replace(True, '위험')
  df_r2 = df_r0[df_r0['analysis']==list_inverse[2].replace(False, '양호')]
  df_r3 = df_compare[df_compare['analysis']==list_inverse[3]].replace(True, '위험')
  df_r3 = df_r0[df_r0['analysis']==list_inverse[3].replace(False, '양호')]
  df_r4 = df_compare[df_compare['analysis']==list_inverse[4]].replace(True, '위험')
  df_r4 = df_r0[df_r0['analysis']==list_inverse[4].replace(False, '양호')]
  df_inverse = pd.concat([df_r0, df_r1, df_r2, df_r3, df_r4], axis=0, ignore_index = True)

  df_compare = df_compare.replace(True, '양호')
  df_compare = df_compare.replace(False, '위험')

  idx_nm_1 = df_compare[df_compare['analysis']==list_inverse[0]].index
  df_compare = df_compare.drop(idx_nm_1)
  idx_nm_2 = df_compare[df_compare['analysis']==list_inverse[1]].index
  df_compare = df_compare.drop(idx_nm_2)
  idx_nm_3 = df_compare[df_compare['analysis']==list_inverse[2]].index
  df_compare = df_compare.drop(idx_nm_3)
  idx_nm_4 = df_compare[df_compare['analysis']==list_inverse[3]].index
  df_compare = df_compare.drop(idx_nm_4)
  idx_nm_5 = df_compare[df_compare['analysis']==list_inverse[4]].index
  df_compare = df_compare.drop(idx_nm_5)

  df_condition = pd.concat([df_inverse, df_compare], axis=0, ignore_index=True)
  df_condition['corp_code'].fillna(f'{corp_code}', inplace=True)
  df_condition['analysis'].fillna('차입금평균이자율', inplace=True)
  return df_condition

def nan_group(values):
  if values == '양호'
    return ''

  if values == '위험'
    return ''

def nan_filtering(df):
  df_npresent = df[df['present'].isnull()]
  df_nprior = df[df['prior'].isnull()]
  df_npriorprior = df[df['priorprior'].isnull()]

  df_npresent['present_condition']=df_npresent['present_condition'].apply(nan_group)
  idx_npresent = df_npresent['present_condition'].index
  df_npresent['prior_condition']=df_nprior['prior_condition'].apply(nan_group)
  idx_nprior = df_nprior['prior_condition'].index
  df_npresent['priorprior_condition']=df_npriorprior['priorprior_condition'].apply(nan_group)
  idx_npriorprior = df_npriorprior['priorprior_condition'].index

  df_nidx = df.drop(idx_npresnet)

  df_reseult_condition = pd.concat([df_nidx, df_present], axis = 0, ignore_index=True)

  return df_result_condition

def _load_corp_code():
  mysqldb = pymysql.connect(host='localhost', user='root', password='pw', df='qw_test', charset='utf8mb4')
  cursor = mysqldb.cursor(pymysql)

  corp_code = f"select distinct 고유번호 from qw_test.bs_timeseries"

  cursor.execute(corp_code)
  result_corp_code = cursor.fetchall()
  df_corp_code = pd.DataFrame(result_corp_code)

def post_condition(df):

  pymysql.install_as_MySQLdb()
  engine = create_engine("mysql+mysqldb://root:pw@localhost:3306/qw_test", encoding='utf-8')
  conn = engine.connect()
  df.to_sql(name='table_name', con=engine, if_exists='append', index=False)


In [ ]:
corp_code = _load_corp_code()

In [ ]:
condition_data(corp_code[i])

In [ ]:
nan_filtering(condition_data(corp_code[i]))

In [ ]:
for i in range(len(corp_code)):
  post_condition(nan_filtering(condition_data(corp_code)))